<h1>Modelo de classificação do gênero do filme</h1>

<h2>Criação do dataset e definição dos atributos</h2>

<html>
<head>
    <title><h2><b>Competição - Classificador de Filmes</title>
    <meta charset="utf-8"/>
</head>
<body>
Inicialmente efetuou-se a leitura do dataset de treino. O Dataset lido foi armazenado no dataframe df_amostra. Para que pudessemos efetuar
o treinamento, os dados dos filmes precisaram passar por uma etapa de tratamento.
Como cada filme possui dados de atores, escritores, diretores, ano, popularidade e etc, e muitos destes dados não possuem valor agregado ou
mesmo não são relevantes para nosso caso de estudo. Partimos das seguintes premissas para a escolha dos atributosinstancias:

- Atores:  muitos atores participam de filmes dentro de um mesmo gênero. Por exemplo: Adan Sandler é conhecido por fazer filmes de comédia. Binarizamos os dados dos atores. Atores cuja presença foi detectada em pelo menos 3 instâncias de filme será considerado um atributo relevante.

- Diretor: Este atributo segue a mesma premissa do atributo ator.

- Data de estreia: este atributo foi considerado relevante por acreditarmos que cada época possui um tipo de filme que é mais produzido. Exemplo: após 2008 foram produzidos um número muito grande de filmes de ação pela Marvel. Este dado foi tratado de modo a apresentar apenas o ano no dataframe.

Tentamos utilizar o parâmetro resumo utilizando ```BagOfWords```, no entanto, devido a quantidade elevada de colunas geradas ocorreu estouro de memória. Portanto, decidimos que iríamos fazer um dataset separado contendo apenas o resumo processado devidamente.

Após o tratamento dos atributos para os dois datasets, foram realizados os experimentos.
</body>
</html>

In [1]:
import pandas as pd
import optuna
import pickle
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from base_am.resultado import Fold
from base_am.avaliacao import Experimento
from base_am.preprocessamento_atributos import BagOfItems, BagOfWords
from competicao_am.metodo_competicao import MetodoCompeticao
from competicao_am.avaliacao_competicao import *
from competicao_am.gerar_resultado_teste import *
from competicao_am.preprocessamento_atributos_competicao import *

df_amostra = pd.read_csv("datasets/movies_amostra.csv")

bag_cast = BagOfItems(min_occur = 4)
df = gerar_atributos(df = df_amostra,bag_cast = bag_cast)
df

0/3000
1000/3000
2000/3000


,Vittorio Gassman,Samuel L. Jackson,Cantinflas,Jean-Claude Van Damme,Richard Pryor,Fred Williamson,Renato Pozzetto,Brendan Gleeson,Paul Reubens,William Hurt,...,Jay Chapman,John Woo,Billy Wilder,Robert Altman,Robert Stevenson,Tsui Hark,Juliusz Machulski,id,data_de_estreia,genero
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,86295,1981,2
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,289198,2014,2
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,24382,1958,1
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,479,2000,2
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,37712,2001,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,52448,1978,1
2996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,26603,1988,1
2997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,45438,2000,1
2998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,114060,1958,1


In [2]:
bow = BagOfWords()
df_resumo = bow.cria_bow(df_amostra,"resumo")
df_resumo["genero"] = df["genero"]
df_resumo

,00,000,007,009,05pm,10,100,1000,1001,100th,...,światłem,şehirlere,şimdi,şte,że,życie,życiowe,आव,गल,genero
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


<h2>Experimentação com métodos de classificação</h2>

<html>
<body>
Com os dados tratados, foram realizados os experimentos e análise dos métodos e métricas. Identificamos o problema como um caso de Classificação. Como métodos avaliados utilizamos:

- LinearSVC: implementa um algoritmo de classificação, que encontra uma linha que separa as classes. O parâmetro random_state indica a seed para embaralhar os dados (0 não embaralha). Definimos o valor como 2.

- DecisionTreeClassifier: método não paramétrico de aprendizagem supervisionada. O parâmetro min_samples_split indica o número mínimo de amostras necessárias para separar um nó interno enquanto que random_state indica a seed para embaralhar os dados. (ver https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)

- RandomForestClassifier: é um estimador que ajusta um número de classificadores de árvores de decisão em sub-amostras do dataset e utiliza  a média para melhorar a precisão da predição e controla o overfitting. O parâmetro n_estimators define o número de árvores, max_features indica o número de features a serem considerados ao procurar a melhor divisão, min_samples_split indica o número mínimo de amostras necessárias para separar um nó interno e random_state indica a seed para embaralhar os dados (2). (ver https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

- AdaBoostClassifier: é um método que combina vários classificadores de baixo desempenho para obter um classificador forte de alta precisão. São atribuídos pesos aos classificadores e o treino é realizado em cada iteração, onde ele tenta fornecer um ajuste excelente, minimizando o erro de treinamento. O parâmetro learning_rate = 1 define a taxa de aprendizado - quanto menor o valor, melhor costuma ser o resultado e maior o tempo.

- ExtraTreeClassifier: é um classificador de árvore extremamente aleatório. "Ao procurar a melhor divisão para separar as amostras de um nó em dois grupos, divisões aleatórias são desenhadas para cada max_features" e a melhor divisão entre eles é escolhida. Quando max_features é  1, a árvore de decisão é totalmente aleatória (ver https://scikit-learn.org/stable/modules/generated/sklearn.tree.ExtraTreeClassifier.html).

- GradientBoostingClassifier: é um algoritmo que combinam modelos de aprendizado de máquina fracos para criar um modelo preditivo forte, como o AdaBoostingClassifier. O default é utilizar decision Tree para fazer o aumento de gradiente. Em cada etapa, nclasses árvores de regressão são ajustadas no gradiente negativo da função de perda de desvio binomial ou multinomial. (ver https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html)
    
    
Após a execução dos experimentos, seguiu-se com a avaliação das métricas para análise dos métodos que obtiveram os melhores desempenhos para o caso de estudo. Com isto feito, os melhores valores de parâmetros foram adotados.
</body>
</html>

<h2>Análise dos resultados e Conclusão</h2>

Nós utilizamos o ```pickle``` para salvar os experimentos no arquivos e se tornar de forma mais rápida a visualização deles (o que pode ser visto abaixo). Além disso criamos um dataframe com as metricas para avaliar o melhor método dentre os 6 que foram experimentados.

In [3]:
#caso queira ver nossos resultados para os experimentos retire o comentario
gerar_resultados_df_cast()

,Métricas,GradientBoostingClassifier,RandomForest,ExtraTreeClassifier,AdaBoostClassifier,DecisionTreeClassifier,Linear_SVC
0,acuracia_media,0.736333,0.769333,0.733,0.708,0.706,0.528
1,precisao_media,"[0.7458742797019504, 0.6742520055995262]","[0.7908376203076495, 0.7257836294405487]","[0.7309806716218554, 0.6869503704516132]","[0.731233544706915, 0.7121367702857169]","[0.7099642115510538, 0.6354948363313293]","[0.49028275256635834, 0.44953110536787816]"
2,revocacao_media,"[0.7167264250844403, 0.6120980304245798]","[0.7428210988470315, 0.6578362119838398]","[0.737950933018791, 0.5654599943099177]","[0.6771310423218238, 0.4899716282843781]","[0.6662994342130661, 0.6011122569812678]","[0.3230559482960289, 0.6884491100455199]"
3,f1_por_classe_media,"[0.7189933116660747, 0.6280867542830515]","[0.7509673910363566, 0.6757067340845786]","[0.7216312604203111, 0.5969589679997709]","[0.6406074012537104, 0.49045181364192986]","[0.6722842565956512, 0.6034784929624104]","[0.36970999070973676, 0.5280902721761584]"
4,Macro F1,0.67354,0.713337,0.659295,0.56553,0.637881,0.4489


Considerando um dataset formado unicamente pelo elenco, diretores e a data de estreia do filme obtivemos os resultados acima. Pela análise dos resultados das métricas, observamos que o método que obteve os melhores resultados para a métrica Macro F1 foi o Random Forest. A Macro F1 apresenta a média harmônica entre a revocação e a precisão, indicando um valor balanceado para as classes. 
Nossos resultados indicaram uma revocação um tanto desequilibrada entre as classes na maioria dos métodos, ou seja, uma classe tem maior facilidade de ser prevista que a outra, possivelmente em decorrência da existência de um número maior de instâncias pertencentes a uma classe em relaçao a outra, criando um viés de tendência.
Lembrando que foram realizadas 30 tentativas para todos os métodos utilizando o algoritmo TPE sampler para escolher parâmetros que tenham resultados melhores. Aqueles que utilizam learning_rate, utilizamos valores de 0 a 1, min_samples_split de 0 a 0,5, max_features de 0 a 0,5 e o número de árvores de 1 a 100. Além disso, o método LinearSVC utilizou um valor C variando de 10^-5 a 10^5.
Sendo assim, para este dataset, o melhor metódo foi o RandomForest. No entanto, fizemos uma segunda rodada de testes utilizando um dataset contendo apenas o resumo (utilizando da classe BagOfWords). Para tratar melhor os dados do resumo, removemos as palavras irrelevantes como artigos, pronomes, advérbios, entre outros. Para isso fizemos uma pesquisa procurando o maior conjunto possível de palavras desse tipo no idioma inglês utilizando o site: https://countwordsfree.com/stopwords.

In [4]:
#caso queira ver nossos resultados para os experimentos retire o comentario
gerar_resultados_df_resumo()

,Métricas,AdaBoostClassifier,Linear_SVC
0,acuracia_media_Folds,0.808667,0.809
1,precisao_media_Folds,"[0.82232642357649, 0.7579140840446128]","[0.8143701340475534, 0.7708187347089507]"
2,revocacao_media_Folds,"[0.7900559293665496, 0.7685206826862406]","[0.801722333765823, 0.7359738164817508]"
3,f1_por_classe_media_Folds,"[0.8046183887928979, 0.7606196762470152]","[0.8049844199168792, 0.7499030211938327]"
4,Macro F1,0.782619,0.777444


Observamos que os resultados obtidos com o dataset composto pelo resumo foram melhores do que o primeiro dataset testado. As métricas de precisão para a classe Comedy e a acurácia foram superiores ao método RandomForest nos métodos LinearSVC e AdaBoosting. Sendo assim optou-se pelo método AdaBoosting.

Após decidirmos sobre o método a ser adotado (Ada Boosting utilizando como base_estimator o LinearSVC), avaliamos dentro do método as melhores métricas. Para isso, a cada iteração (cada fold), avaliamos o valor da métrica Macro F1. Com isto, a iteração com o maior valor é avaliada separadamente e escolhemos os parâmetros da trial com os melhores resultados.

<h2>Analise de resultados da segunda rodada</h2>

<html>
<body>
Após os resultado da primeira rodada, optamos por realizar uma segunda análise dos métodos e dos atributos de interesse. Mais uma vez, com base nos nossos resultados, o resumo se mostrou a melhor opção. Partindo desse ponto analisamos como seria o resultado se uníssemos dois métodos distintos, a saber: AdaBoost Classifier e Linear SVC. Para tal, adotamos o seguinte critério:
    
- Caso o AdaBoost Classifier classificasse o filme como comédia, este seria o valor adotado. Se a classificação fosse ação adotaríamos o LinearSVC.

Esta foi uma tentativa de minizarmos os valores de revocação mais baixos do AdaBoost para ação. No entanto não obtivemos um resultado satisfatório. Muito pelo contrário os resultados ficaram abaixo do esperado. Com isso, resolvemos analisar os paramêtros obtidos pelo método AdaBoost, que fora escolhido como melhor na primeira rodada. Dessa forma, analizamos o best_params de cada fold e fixamos este valor na OtimizacaoObjetivo. Notamos que apesar do fold 5 ter tido o melhor Macro F1, ao rodarmos o teste, os resultados foram menores do que os resultados dos demais folds. Então optamos pelo fold 2 que apesar de ter tido um Macro F1 um pouco menor que o do fold 5, no teste obteve um resultado mais adequado.
</body>
</html>

In [5]:
import pickle

exp_adaElinear = pickle.load( open( "resultados.exp_AdaBoostELinear_resumo_1.p", "rb" ) )
acuracia_media_AdaBoost, precisao_media_AdaBoost, revocacao_media_AdaBoost, f1_classe_media_AdaBoost = Calculos.calcula_indices_importantes(exp_adaElinear.resultados)
df = pd.DataFrame({
    'Métricas' : ["acuracia_media_Folds", "precisao_media_Folds", "revocacao_media_Folds", "f1_por_classe_media_Folds", "Macro F1"],
    'AdaBoostClassifier e Linear' : [acuracia_media_AdaBoost, precisao_media_AdaBoost, revocacao_media_AdaBoost, f1_classe_media_AdaBoost, exp_adaElinear.macro_f1_avg]
})

df

,Métricas,AdaBoostClassifier e Linear
0,acuracia_media_Folds,0.793333
1,precisao_media_Folds,"[0.8305593506288927, 0.7128051262624142]"
2,revocacao_media_Folds,"[0.7487305225390936, 0.7980923485988489]"
3,f1_por_classe_media_Folds,"[0.7862362000382991, 0.752676321435355]"
4,Macro F1,0.769456


<h2>Aplicação do modelo em um dataset de testes</h2>

Para gerar a saída do resultado da amostra de teste iniciamente devemos ler o arquivo .csv, por intermédio da função ```pd.read_csv```. Após isso, é chamada a função ```gerar_saida_teste``` recebendo como parâmetros o dataset e a classe que queremos prever além do número do nosso grupo (no caso o grupo 4). No escopo da função é lido o dataset utilizado para o treino. Depois disso, foi realizado o processamento do resumo utilizando a classe BagOfWords para o dataset de treino e o de testes. Foi instanciado um objeto da classe MetodoCompeticao passando como parâmetro o melhor método de classificação que obtivemos após a realização dos experimentos e com seu parâmetros já fixados (o AdaBoosting). Utilizando o dataset processado do treino e a classe ```genero``` foi feito o ```fit``` e obtido um modelo. Com base nesse modelo foi feito o ```predict``` utilizando o dataset processado de testes. Por fim o vetor de predições foi processado utilizando um dicionário e passado para o arquivo.

In [6]:
df_amostra_teste = pd.read_csv("datasets/movies_amostra_teste_2.csv")
gerar_saida_teste(df_amostra_teste,"genero",4)


FileNotFoundError: [Errno 2] File datasets/movies_amostra_teste_2.csv does not exist: 'datasets/movies_amostra_teste_2.csv'